In [1]:
# Homework 7
# Adrien Masini

In [24]:
# import modules
import numpy as np
import astropy.constants as const
from CenterOfMass import CenterOfMass
from GalaxyMass import ComponentMass
import math
import matplotlib.pyplot as plt

In [39]:
class M33AnalyticOrbit : 
    '''Class whith a series of functions that will determine the acceleration M33
feels from M31 and integrate its current position and velocity forwards in time.
'''
    def __init__(self, filename):
    # Initialize the instance of this Class with the following properties:
    
 #Defining the value of G       
        #self.G = const.G.to(u.kpc**3/u.Msun/u.Gyr**2).value
        self.G = 4.498502151575286e-06
        
 #Store the Filename :
        self.filename = filename
        
# Determine the relative position and velocity between M33 and M31

        self.M31_COM = CenterOfMass("M31_000.txt", 2)
        self.M33_COM = CenterOfMass("M33_000.txt", 2)
        
        # M31:   store the position and velocity COM
        self.M31_COM_P = self.M31_COM.COM_P(0.1) #.value return AttributeError: 'numpy.ndarray' object has no attribute 'value'
        self.M31_COM_V = self.M31_COM.COM_V(self.M31_COM_P[0],self.M31_COM_P[1],self.M31_COM_P[2]) #.value return AttributeError: 'numpy.ndarray' object has no attribute 'value'

        # M33:   store the position and velocity COM
        self.M33_COM_P = self.M33_COM.COM_P(0.1) #.value return AttributeError: 'numpy.ndarray' object has no attribute 'value'
        self.M33_COM_V = self.M33_COM.COM_V(self.M33_COM_P[0],self.M33_COM_P[1],self.M33_COM_P[2]) #.value return AttributeError: 'numpy.ndarray' object has no attribute 'value'
#Define the vectors : 
        self.M33_M31 = (self.M33_COM_P - self.M31_COM_P) #.value return AttributeError: 'numpy.ndarray' object has no attribute 'value'
        #self.vM33_M31 = self.M33_COM_V - self.M31_COM_P #Works but not correct
        #self.vM33_M31 = self.M33_COM_V - self.M31_COM_V #TypeError: unsupported operand type(s) for -: 'list' and 'list'
        self.Vx = self.M33_COM_V[0] - self.M31_COM_V[0]
        self.Vy = self.M33_COM_V[1] - self.M31_COM_V[1]
        self.Vz = self.M33_COM_V[2] - self.M31_COM_V[2]
        self.vM33_M31 = np.array([self.Vx,self.Vy, self.Vz])
        
#The scale lengths and masses for each component in M31
        self.rdisk = 5
        self.Mdisk = ComponentMass('M31_000.txt', 2)*1e12
          
        self.rbulge = 1
        self.Mbulge = ComponentMass('M31_000.txt', 3)*1e12 
    
        self.rhalo = 62
        self.Mhalo = ComponentMass('M31_000.txt', 1)*1e12
        
        
#2.1 Halo and Bulge Acceleration

        #Acceleration term
    def HenquistAccel(self, M, ra, x, y, z):
    
            r = math.sqrt((x**2)+(y**2)+(z**2))
            rvect = np.array([x,y,z])
            aH = -self.G*M/(r*(ra+r)**2 )*rvect
            return aH
        
#2.2 Disk Acceleration
        
        
    def MiyamotoNagaiAccel(self, M, rd, x, y, z) :
            
            R = math.sqrt((x**2)+(y**2))
            rd = self.rdisk
            zd = self.rdisk/5.0
            B = rd + math.sqrt((z**2)+(zd**2))
            rvec = np.array([x,y,z])
            aM = -self.G*M/((R**2)+B**2)**1.5*np.array([1,1,B/(B-rd)])*rvec
            
            return aM
        
#2.3 M31 Acceleration

    def M31Accel(self, r) :
            
            aHH = self.HenquistAccel(self.Mhalo,self.rhalo,r[0],r[1],r[2])
            aHB = self.HenquistAccel(self.Mbulge,self.rbulge,r[0],r[1],r[2])
            aM = self.MiyamotoNagaiAccel(self.Mdisk,self.rdisk,r[0],r[1],r[2])
    
            aTot = aHH+aHB+aM
            
            return aTot
            
#3 Build an Integrator
        
    def LeapFrog(self, dt, r, v):
            
            # Update the position 
            rh = r + v*dt/2 
            
            # Update the velocity
            vnew = v + self.M31Accel(rh)*dt
            
            # Store the new position
            rnew = r + 0.5*(vnew + v)*dt
            
            return rnew, vnew
            
#4 Integrate the Orbit            
            
    def OrbitIntegrator(self, t0, dt, tmax) :
        
        
        Rinit = self.M33_M31 #np.array(M33_M31)
        Vinit = self.vM33_M31 #np.array(vM33_M31)
        
        t = t0
        orbit = np.zeros([int(tmax/dt)+2, 7])
        orbit[0] = t0, Rinit[0], Rinit[1], Rinit[2], Vinit[0], Vinit[1], Vinit[2]

        i = 1
        print(i)
        print(orbit)  
        
        while t < tmax :

                t += dt
                orbit[i,0] = t

                orbit[i,1:4], orbit[i,4:] = self.LeapFrog(dt, orbit[i-1,1:4], orbit[i-1,4:])
                
                i += 1
                
                print(i)
                print(orbit)
                
            
        np.savetxt(self.filename, orbit, fmt = "%11.3f"*7, comments='#', 
               header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                   .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))
                      


#grp = M33AnalyticOrbit("M33_000.txt")
grp = M33AnalyticOrbit("orbit.txt")
line = grp.OrbitIntegrator(0,0.1,10) 


1
[[   0.          -98.56       -119.99       -127.76        -28.4241337
   173.92565128   93.22761337]
 [   0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.            0.            0.      

UnitsError: Can only apply 'add' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)

In [ ]:
#Plot the graph :
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111)
plt.plot(line, color='blue', linewidth=4, label='Prediction')

# Add axis labels
plt.xlabel('', fontsize=22)
plt.ylabel('', fontsize=22)

label_size = 22
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

# Legend
legend = ax.legend(loc='upper right',fontsize='x-large')